## Silverbox

Silverbox is the name of a nonlinear system identification benchmark, proposed in 2004. Data, baselines and more info can be found at http://nonlinearbenchmark.org/#Silverbox.

State-space formulation of Silverbox's dynamics:

$$\begin{align}
\mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + \kappa(x(t)) x(t) =&\ u(t) + w(t) \\
\kappa(x(t)) =&\ \alpha + \beta x^2(t) \\
y(t) =&\ x(t) + e(t)
\end{align}$$

where
$$\begin{align}
\mu     =&\ \text{mass} \\
\nu     =&\ \text{viscous damping} \\
\kappa(x(t)) =&\ \text{nonlinear spring} \\
y(t)    =&\ \text{observation (displacement)} \\
x(t)    =&\ \text{state (displacement)} \\
u(t)    =&\ \text{force} \\
e(t)    =&\ \text{measurement noise} \\
w(t)    =&\ \text{process noise}
\end{align}$$

### Steps to solve

I now take a series of steps to re-write this problem:

#### 1. Assume constant spring coefficient κ

$$ \mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + κ x(t) = u(t) + w(t)$$

#### 2. Divide by leading coefficient

$$ \frac{d^2 x(t)}{dt^2} + \frac{\nu}{\mu} \frac{d x(t)}{dt} + \frac{κ}{\mu} x(t) = \frac{1}{\mu} u(t) + \frac{1}{\mu} w(t)$$

#### 3. Substitute standard variables

$$ \frac{d^2 x(t)}{dt^2} + 2\zeta \omega_0 \frac{d x(t)}{dt} + \omega_0^2 x(t) - \frac{u(t)}{\mu} = \frac{w(t)}{\mu}$$

where $$\begin{align} 
\zeta =&\ \frac{\nu}{2\sqrt{\mu \kappa}} \\ 
\omega_0 =&\ \sqrt{\frac{\kappa}{\mu}} \, .
\end{align}$$

#### 4. Apply Euler's method to obtain difference equation (step size is 1)

-> Forward Euler:

$$\begin{align}
\frac{x(t+2h)-2x(t+h)+x(t)}{h^2} + 2\zeta \omega_0 \frac{x(t+h)-x(t)}{h} + \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t+2) + 2(\zeta \omega_0 - 1) x(t+1) + (1 - 2 \zeta \omega_0 + \omega_0^2) x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} 
\end{align}$$

-> Backward Euler:

$$\begin{align}
\frac{x(t)-2x(t-h)+x(t-2h)}{h^2} - 2\zeta \omega_0 \frac{x(t)-x(t-h)}{h} - \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu}  \\
(1 + 2 \zeta \omega_0 + \omega_0^2)x(t) - 2(1 + \zeta \omega_0)x(t-1) - x(t-2) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t) - \frac{2(1 + \zeta \omega_0)}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-1) - \frac{1}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-2) - \frac{u(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)} =&\ \frac{w(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)}
\end{align}$$
    
Change to shorthand notation:

$$x_t - \alpha x_{t-1} - \beta x_{t-2} - \gamma u_t = \gamma w_t$$

where 
$$\begin{align} 
\alpha =&\ \frac{2(1 + \zeta \omega_0)}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\beta =&\ \frac{1}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\gamma =&\ \frac{1}{\mu(1 + 2 \zeta \omega_0 + \omega_0^2)}
\end{align}$$

#### 5. Convert to multivariate first-order difference form

Stick to backward Euler (matches AR structure)
- Backward Euler:

    $$z_t = M z_{t-1} + N u_t + N w_t$$

    where $z_t = [x_t\ \ x_{t-1}]$, $M = [α \ \ β; 1\ \ 0]$, $N = [γ\ \ 0]$

#### 6. Convert to Gaussian probability

- Backward Euler:

$$z_t \sim \mathcal{N}(M z_{t-1} + N u_t, N \tau)$$

where $w_t \sim \mathcal{N}(0, \tau)$

#### 7. Observation likelihood

$$y_t \sim \mathcal{N}(c z_t, σ)$$

where $e_t \sim \mathcal{N}(0, \sigma)$, $c = [1\ \ 0]$

Now, I need priors for $\alpha$, $\beta$, $\gamma$, $\tau$, $\sigma$. Given three equations and three unknowns, I can recover $\zeta$, $\omega_0$ and $\mu$ from $\alpha$, $\beta$, and $\gamma$. The variables are all strictly positive, which means they can be modeled by gamma distributions:

$$\begin{align}
\alpha \sim&\ Γ(1, 1e3) \\
\beta \sim&\ Γ(1, 1e3) \\
\gamma \sim&\ Γ(1, 1e3) \\
\tau \sim&\ Γ(1, 1e3) \\
\sigma \sim&\ Γ(1, 1e3) 
\end{align}$$

--> Implementation with ForneyLab and AR node

In [1]:
# Generate time-series
using CSV
using DataFrames
using Plots

# Read data from CSV file
df = CSV.read("../data/SNLS80mV.csv", ignoreemptylines=true)
df = select(df, [:V1, :V2])

# Shorthand
input = df[:,1]
output = df[:,2]

# Time horizon
T = size(df, 1)

131072

In [59]:
using ForneyLab
using LAR
using LAR.Node, LAR.Data
using ProgressMeter

I will introduce another shorthand: $\theta = [\alpha\ \ \beta]$ and use the Nonlinear node to provide the AR node with a Gaussian form for $\theta$.

In [77]:
# Start graph
graph = FactorGraph()

# Static parameters
@RV log_θ ~ GaussianMeanPrecision(placeholder(:m_θ_t, dims=(2,)), placeholder(:w_θ_t, dims=(2,2)))
@RV τ ~ Gamma(placeholder(:a_τ_t), placeholder(:b_τ_t))

# Nonlinear function
g(x) = exp.(x)
g_inv(x) = log.(x)

# Nonlinear node
# @RV θ ~ Nonlinear(log_θ; g=g, g_inv=g_inv, dims=(2,))
@RV θ ~ Nonlinear(log_θ; g=g, dims=(2,))

# I'm fixing measurement noise σ
σ = 10.

# Observation selection variable
c = [1, 0]

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z_t, dims=(2,)), placeholder(:w_z_t, dims=(2, 2)))

# Autoregressive node
@RV x_t ~ Autoregressive(θ, z_t, τ)

# Specify likelihood
@RV y_t ~ GaussianMeanPrecision(dot(c, x_t), σ)

# Placeholder for data
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(graph)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 2570193667581509431 -->
 
 2570193667581509431 
 
 placeholder_w_z_t 
 
<!-- 13950325492396342969 -->
 
 13950325492396342969 
 
 dot 
 dotproduct_1 
 
<!-- 7406626485671933878 -->
 
 7406626485671933878 
 
 clamp_1 
 
<!-- 13950325492396342969--7406626485671933878 -->
 
 13950325492396342969--7406626485671933878 
 
 clamp_1 
 1 out 
 3 in2 
 
<!-- 5683426925324319582 -->
 
 5683426925324319582 
 
 AR 
 autoregressive_1 
 
<!-- 13950325492396342969--5683426925324319582 -->
 
 13950325492396342969--5683426925324319582 
 
 x_t 
 1 y 
 2 in1 
 
<!-- 2359882635262545797 -->
 
 2359882635262545797 
 
 placeholder_a_τ_t 
 
<!-- 3874193723918675476 -->
 
 3874193723918675476 
 
 Nonlinear{Unscented} 
 nonlinear_1 
 
<!-- 158871959477260110 -->
 
 158871959477260110 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 3874193723918675476--158871959477260110 -->
 
 3874193723918675476--158871959477260110 
 
 log_θ 
 1 out 
 2 in1 
 
<!-- 10431455697477631642 -->
 
 10431455697477631642 
 
 placeholder_m_z_t 
 
<!-- 9356182925043428520 -->
 
 9356182925043428520 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 9356182925043428520--2570193667581509431 -->
 
 9356182925043428520--2570193667581509431 
 
 w_z_t 
 1 out 
 3 w 
 
<!-- 9356182925043428520--10431455697477631642 -->
 
 9356182925043428520--10431455697477631642 
 
 m_z_t 
 1 out 
 2 m 
 
<!-- 11058035638927901572 -->
 
 11058035638927901572 
 
 placeholder_b_τ_t 
 
<!-- 5683426925324319582--3874193723918675476 -->
 
 5683426925324319582--3874193723918675476 
 
 θ 
 1 out 
 3 θ 
 
<!-- 5683426925324319582--9356182925043428520 -->
 
 5683426925324319582--9356182925043428520 
 
 z_t 
 1 out 
 2 x 
 
<!-- 15450970219322370840 -->
 
 15450970219322370840 
 
 Gam 
 gamma_1 
 
<!-- 5683426925324319582--15450970219322370840 -->
 
 5683426925324319582--15450970219322370840 
 
 τ 
 1 out 
 4 γ 
 
<!-- 6042170533216002331 -->
 
 6042170533216002331 
 
 placeholder_m_θ_t 
 
<!-- 158871959477260110--6042170533216002331 -->
 
 158871959477260110--6042170533216002331 
 
 m_θ_t 
 1 out 
 2 m 
 
<!-- 14208128379434238716 -->
 
 14208128379434238716 
 
 placeholder_w_θ_t 
 
<!-- 158871959477260110--14208128379434238716 -->
 
 158871959477260110--14208128379434238716 
 
 w_θ_t 
 1 out 
 3 w 
 
<!-- 17606185177036470806 -->
 
 17606185177036470806 
 
 clamp_2 
 
<!-- 12310434417067870059 -->
 
 12310434417067870059 
 
 placeholder_y_t 
 
<!-- 2514183915610078051 -->
 
 2514183915610078051 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 12310434417067870059--2514183915610078051 -->
 
 12310434417067870059--2514183915610078051 
 
 y_t 
 1 out 
 1 out 
 
<!-- 15450970219322370840--2359882635262545797 -->
 
 15450970219322370840--2359882635262545797 
 
 a_τ_t 
 1 out 
 2 a 
 
<!-- 15450970219322370840--11058035638927901572 -->
 
 15450970219322370840--11058035638927901572 
 
 b_τ_t 
 1 out 
 3 b 
 
<!-- 2514183915610078051--13950325492396342969 -->
 
 2514183915610078051--13950325492396342969 
 
 variable_1 
 1 out 
 2 m 
 
<!-- 2514183915610078051--17606185177036470806 -->
 
 2514183915610078051--17606185177036470806 
 
 clamp_2 
 1 out 
 3 w

In [78]:
# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, θ, τ, ids=[:z, :x, :θ, :τ])
algo = variationalAlgorithm(q, free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
eval(Meta.parse(source_code));
println(source_code)

begin

function stepτ!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGammaOut(nothing, ProbabilityDistribution(Univariate, PointMass, m=data[:a_τ_t]), ProbabilityDistribution(Univariate, PointMass, m=data[:b_τ_t]))
messages[2] = ruleVariationalARIn3PPPN(marginals[:x_t], marginals[:z_t], marginals[:θ], nothing)

marginals[:τ] = messages[1].dist * messages[2].dist

return marginals

end

function stepz!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGaussianMeanPrecisionOut(nothing, ProbabilityDistribution(Multivariate, PointMass, m=data[:m_z_t]), ProbabilityDistribution(MatrixVariate, PointMass, m=data[:w_z_t]))
messages[2] = ruleVariationalARIn1PNPP(marginals[:x_t], nothing, marginals[:θ], marginals[:τ])

marginals[:z_t] = messages[1].dist * messages[2].dist

return marginals

end

function initθ()

messages = Array{Message}(undef, 4)

messages[1] = Message(vague

In [79]:
# Inference parameters
num_iterations = 10

# Initialize marginal distribution and observed data dictionaries
data = Dict()
marginals = Dict()

# Initialize arrays of parameterizations
params_x = (zeros(2,T+1), repeat(float(eye(2)), outer=(1,1,T+1)))
params_z = (zeros(2,T+1), repeat(float(eye(2)), outer=(1,1,T+1)))
params_θ = (zeros(2,T+1), repeat(float(eye(2)), outer=(1,1,T+1)))
params_τ = (ones(1,T+1), ones(1,T+1))

# Start progress bar
p = Progress(T, 1, "At time ")

# Perform inference at each time-step
for t = 1:T

    # Update progress bar
    update!(p, t)

    # Initialize marginals
    marginals[:x_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_x[1][:,t], w=params_x[2][:,:,t])
    marginals[:z_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_z[1][:,t], w=params_z[2][:,:,t])
    marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_θ[1][:,t], w=params_θ[2][:,:,t])
#     marginals[:θ] = ProbabilityDistribution(Multivariate, SampleList; s=[0., 0.], w=[1. 0.;0. 1.], diff_ent=nothing)
#     marginals[:θ] = vague(SampleList, 2)
    marginals[:τ] = ProbabilityDistribution(Univariate, Gamma, a=params_τ[1][1,t], b=params_τ[2][1,t])
    
    data = Dict(:y_t => output[t],
                :m_z_t => params_z[1][:,t],
                :m_θ_t => params_θ[1][:,t],
                :w_z_t => params_z[2][:,:,t],
                :w_θ_t => params_θ[2][:,:,t],
                :a_τ_t => params_τ[1][1,t],
                :b_τ_t => params_τ[2][1,t])

    # Iterate variational parameter updates
    for i = 1:num_iterations

        stepx!(data, marginals)
        stepz!(data, marginals)
        stepθ!(data, marginals)
        stepτ!(data, marginals)
    end

    # Store current parameterizations of marginals
    params_x[1][:,t+1] = mean(marginals[:x_t])
    params_z[1][:,t+1] = mean(marginals[:z_t])
    params_θ[1][:,t+1] = mean(marginals[:θ])
    params_x[2][:,:,t+1] = marginals[:x_t].params[:w]
    params_z[2][:,:,t+1] = marginals[:z_t].params[:w]
    params_θ[2][:,:,t+1] = marginals[:θ].params[:w]
    params_τ[1][1,t+1] = marginals[:τ].params[:a]
    params_τ[2][1,t+1] = marginals[:τ].params[:b]

end

ErrorException: PosDefException: Matrix is not positive-definite, even after regularization. Array{Float64,2}:
[8.346504266211588e11 2.8301049910417285e9; 2.8301049910417285e9 9.596144633588245e6]